In [1]:
import numpy as np
import steepest_descent_bcktrck as sdb
from steepest_descent_bcktrck import *
import cgm_pol_rib as cgmpb
from cgm_pol_rib import *
import functions as funcs
from sklearn.model_selection import ParameterGrid
from importlib import reload

In [2]:
reload(funcs)
reload(sdb)
reload(cgmpb)

<module 'cgm_pol_rib' from '/home/claudio/Documents/GitHub/Homework-Numerical-Optimisation-/Homework Numerical Optimisation/Unconstrained Optimisation/cgm_pol_rib.py'>

Evaluate rate of convergence

In [3]:
def rate_of_convergence(x_seq: np.ndarray) ->np.ndarray:
    N = 100
    n = x_seq.shape[1]
    p = np.empty(N)
    for k in range(1, N+1):
        if (x_seq[k+2, :] - x_seq[k+1, :]).any() != 0 and (x_seq[k+1, :] - x_seq[k, :]).any() != 0 and (x_seq[k, :] - x_seq[k-1, :]).any() != 0:
            p[k-1] = log(np.linalg.norm(x_seq[k+2, :] - x_seq[k+1, :], 2) / (np.linalg.norm(x_seq[k+1, :] - x_seq[k, :], 2))) / \
                     log(np.linalg.norm(x_seq[k+1, :] - x_seq[k, :], 2) / (np.linalg.norm(x_seq[k, :] - x_seq[k-1, :], 2)))
            # print(f"i = {k-1}: p = {p}")
    return p

Initialisation of the data

In [4]:
x0 = np.array([1.2, 1.2])
x1 = np.array([-1.2, 1])
alpha0 = 1
tolgrad = 1e-12
rho = 0.5
c = 1e-4
kmax = 5000
btmax = 50
fin_diff = False
fd_type = 'centered'

params = {"c": [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6],
              "rho": [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]}

Let's test the Steepest descent and conjugate gradient method with Backtrack tuning the parameters c and rho using the Armijo condition to see what is the result obtained (POINT X0)

Here our idea is quite simple. In order to make a complete comparison between the two methods the best set of paramenters for both methods is evaluated, once this is done a comparison is made with the best result obtained in one of the two methods with one set and the same set is also used for the other method (we will therefore have in all 2 saved evaluations for steepest descent, one with the best set of parameters, and the other with the best set of parameters of the conjugate gradient method, and 2 saved evaluations for the conjugate gradient method) 


In [ ]:
#best result for steepest descent method is saved here
sd_best_fk_x0 = 1
sd_best_param_x0 = 0
sd_best_k_x0 = 0
sd_best_gradfk_norm_x0 = 10
sd_best_xk_x0 = np.empty(2)

#best result for conjugate gradient method is saved here
cg_best_fk_x0 = 1
cg_best_param_x0 = 0
cg_best_k_x0 = 0
cg_best_gradfk_norm_x0 = 10
cg_best_xk_x0 = np.empty(2)

#Other result of steepest descent method is saved here
sd_fk_x0 = 1
sd_param_x0 = 0
sd_k_x0 = 0
sd_xk_x0 = np.empty(2)

#Other result of conjugate gradient method is saved here
cg_fk_x0 = 1
cg_param_x0 = 0
cg_k_x0 = 0
cg__xk_x0 = np.empty(2)

for param in ParameterGrid(params): 
    sd_xk, sd_fk, sd_gradfk_norm, sd_k, sd_x_seq, sd_bt_seq = steepest_descent_bcktrck(x0, 'Rosenbrock', alpha0, kmax, tolgrad, param["c"], param["rho"], btmax, fin_diff, fd_type)

    cg_xk, cg_fk, cg_gradfk_norm, cg_k, cg_x_seq, cg_bt_seq = cgm_pol_rib(x0, 'Rosenbrock', alpha0, kmax, tolgrad, param["c"], param["rho"], btmax, fin_diff, fd_type)
    
    if (sd_fk < sd_best_fk_x0): 
        sd_best_param_x0 = param

        sd_best_xk_x0 = sd_xk
        sd_best_k_x0 = sd_k
        sd_best_fk_x0 = sd_fk
        sd_best_gradfk_norm_x0 = sd_gradfk_norm
        sd_best_x_seq_x0 = np.copy(sd_x_seq)
        
        cg_xk_x0 = cg_xk
        cg_k_x0 = cg_k
        cg_fk_x0 = cg_fk
            
    if (cg_fk < cg_best_fk_x0): 
        cg_best_param_x0 = param

        cg_best_xk_x0 = cg_xk
        cg_best_k_x0 = cg_k
        cg_best_fk_x0 = cg_fk
        cg_best_gradfk_norm_x0 = cg_gradfk_norm
        cg_best_x_seq_x0 = np.copy(cg_x_seq)

        
        sd_xk_x0 = sd_xk
        sd_k_x0 = sd_k
        sd_fk_x0 = sd_fk

In [ ]:
print("Best evaluation with steepest descent method:")
print("x0: ", x0[0], x0[1])
print("best set of parameters: ", sd_best_param_x0)
print("xk: ", sd_best_xk_x0[0], sd_best_xk_x0[1])
print("k: ", sd_best_k_x0)
print("fk: ", sd_best_fk_x0) 

print("\nEvaluation of the conjugate gradient method using the best parameters of steepest descent: ")
print("x0: ", x0[0], x0[1])
print("set of parameters: ", sd_best_param_x0)
print("xk: ", cg_xk_x0[0], cg_xk_x0[1])
print("k: ", cg_k_x0)
print("fk: ", cg_fk_x0) 

Best evaluation with steepest descent method:

x0:  [1.2] [1.2]

best set of parameters:  {'c': 0.01, 'rho': 0.7}

xk:  [1.] [1.]

k:  31839

fk:  [4.17569518e-25]


Evaluation of the conjugate gradient method using the best parameters of steepest descent: 

x0:  [1.2] [1.2]

set of parameters:  {'c': 0.01, 'rho': 0.7}

xk:  [1.] [1.]

k:  8589

fk:  [3.1537594e-25]

In [ ]:
print("Best evaluation with conjugate gradient method:")
print("x0: ", x0[0], x0[1])
print("best set of parameters: ", cg_best_param_x0)
print("xk: ", cg_best_xk_x0[0], cg_best_xk_x0[1])
print("k: ", cg_best_k_x0)
print("fk: ", cg_best_fk_x0) 

print("\nEvaluation of the steepest descent method using the best parameters of conjugate gradient: ")
print("x0: ", x0[0], x0[1])
print("set of parameters: ", cg_best_param_x0)
print("xk: ", sd_xk_x0[0], sd_xk_x0[1])
print("k: ", sd_k_x0)
print("fk: ", sd_fk_x0) 


Best evaluation with conjugate gradient method:

x0:  [1.2] [1.2]

best set of parameters:  {'c': 0.1, 'rho': 0.4}

xk:  [1.] [1.]

k:  330

fk:  [8.72184338e-29]

Evaluation of the steepest descent method using the best parameters of conjugate gradient: 

x0:  [1.2] [1.2]

set of parameters:  {'c': 0.1, 'rho': 0.4}

xk:  [1.] [1.]

k:  27922

fk:  [8.37329555e-25]

In [ ]:
if (sd_best_gradfk_norm_x0 <= tolgrad):
    #It make sense to evaluate the rate of convergence if and only if I've reached the solution 
    p = rate_of_convergence(sd_best_x_seq_x0)
    print("Evaluation of rate of convergence for steepest descent method:")
    print(f"p mean = {np.abs(p).mean():.2f}")

Evaluation of rate of convergence for steepest descent method:

p mean = 1.13

In [ ]:
if (cg_best_gradfk_norm_x0 <= tolgrad):
    #It make sense to evaluate the rate of convergence if and only if I've reached the solution 
    p = rate_of_convergence(cg_best_x_seq_x0)
    print("Evaluation of rate of convergence for conjugate gradient method:")
    print(f"p mean = {np.abs(p).mean():.2f}")

Evaluation of rate of convergence for conjugate gradient method:

p mean = 3.78

Now let's do the same but using another starting point (x1)

In [ ]:
#best result for steepest descent method is saved here
sd_best_fk_x1 = 1
sd_best_param_x1 = 0
sd_best_k_x1 = 0
sd_best_gradfk_norm_x1 = 10
sd_best_xk_x1 = np.empty(2)

#best result for conjugate gradient method is saved here
cg_best_fk_x1 = 1
cg_best_param_x1 = 0
cg_best_k_x1 = 0
cg_best_gradfk_norm_x1 = 10
cg_best_xk_x1 = np.empty(2)

#Other result of steepest descent method is saved here
sd_fk_x1 = 1
sd_param_x1 = 0
sd_k_x1 = 0
sd_xk_x1 = np.empty(2)

#Other result of conjugate gradient method is saved here
cg_fk_x1 = 1
cg_param_x1 = 0
cg_k_x1 = 0
cg__xk_x1 = np.empty(2)

for param in ParameterGrid(params): 
    sd_xk, sd_fk, sd_gradfk_norm, sd_k, sd_x_seq, sd_bt_seq = steepest_descent_bcktrck(x1, 'Rosenbrock', alpha0, kmax, tolgrad, param["c"], param["rho"], btmax, fin_diff, fd_type)

    cg_xk, cg_fk, cg_gradfk_norm, cg_k, cg_x_seq, cg_bt_seq = cgm_pol_rib(x1, 'Rosenbrock', alpha0, kmax, tolgrad, param["c"], param["rho"], btmax, fin_diff, fd_type)
    
    if (sd_fk < sd_best_fk_x0): 
        sd_best_param_x1 = param

        sd_best_xk_x1 = sd_xk
        sd_best_k_x1 = sd_k
        sd_best_fk_x1 = sd_fk
        sd_best_gradfk_norm_x1 = sd_gradfk_norm
        sd_best_x_seq_x1 = np.copy(sd_x_seq)
        
        cg_xk_x1 = cg_xk
        cg_k_x1 = cg_k
        cg_fk_x1 = cg_fk
            
    if (cg_fk < cg_best_fk_x1): 
        cg_best_param_x1 = param

        cg_best_xk_x1 = cg_xk
        cg_best_k_x1 = cg_k
        cg_best_fk_x1 = cg_fk
        cg_best_gradfk_norm_x1 = cg_gradfk_norm
        cg_best_x_seq_x1 = np.copy(cg_x_seq)

        
        sd_xk_x1 = sd_xk
        sd_k_x1 = sd_k
        sd_fk_x1 = sd_fk

In [ ]:
print("Best evaluation with steepest descent method:")
print("x1: ", x1[0], x1[1])
print("best set of parameters: ", sd_best_param_x1)
print("xk: ", sd_best_xk_x1[0], sd_best_xk_x1[1])
print("k: ", sd_best_k_x1)
print("fk: ", sd_best_fk_x1) 

print("\nEvaluation of the conjugate gradient method using the best parameters of steepest descent: ")
print("x1: ", x1[0], x1[1])
print("set of parameters: ", sd_best_param_x1)
print("xk: ", cg_xk_x1[0], cg_xk_x1[1])
print("k: ", cg_k_x1)
print("fk: ", cg_fk_x1) 

Best evaluation with steepest descent method:

x1:  [-1.2] [1.]

best set of parameters:  {'c': 1e-06, 'rho': 0.7}

xk:  [1.] [1.]

k:  35285

fk:  [4.17283211e-25]


Evaluation of the conjugate gradient method using the best parameters of steepest descent: 

x1:  [-1.2] [1.]

set of parameters:  {'c': 1e-06, 'rho': 0.7}

xk:  [1.] [1.]

k:  8191

fk:  [2.04761125e-25]

In [ ]:
print("Best evaluation with conjugate gradient method:")
print("x1: ", x1[0], x1[1])
print("best set of parameters: ", cg_best_param_x1)
print("xk: ", cg_best_xk_x1[0], cg_best_xk_x1[1])
print("k: ", cg_best_k_x1)
print("fk: ", cg_best_fk_x1) 

print("\nEvaluation of the steepest descent method using the best parameters of conjugate gradient: ")
print("x1: ", x1[0], x1[1])
print("set of parameters: ", cg_best_param_x1)
print("xk: ", sd_xk_x1[0], sd_xk_x1[1])
print("k: ", sd_k_x1)
print("fk: ", sd_fk_x1) 


Best evaluation with conjugate gradient method:

x1:  [-1.2] [1.]

best set of parameters:  {'c': 0.01, 'rho': 0.4}

xk:  [1.] [1.]

k:  318

fk:  [5.77643398e-28]


Evaluation of the steepest descent method using the best parameters of conjugate gradient: 

x1:  [-1.2] [1.]

set of parameters:  {'c': 0.01, 'rho': 0.4}

xk:  [1.] [1.]

k:  28146

fk:  [8.1509053e-25]

In [ ]:
if (sd_best_gradfk_norm_x1 <= tolgrad):
    #It make sense to evaluate the rate of convergence if and only if I've reached the solution 
    p = rate_of_convergence(sd_best_x_seq_x1)
    print("Evaluation of rate of convergence for steepest descent method:")
    print(f"p mean = {np.abs(p).mean():.2f}")

Evaluation of rate of convergence for steepest descent method:

p mean = 1.30

In [ ]:
if (cg_best_gradfk_norm_x1 <= tolgrad):
    #It make sense to evaluate the rate of convergence if and only if I've reached the solution 
    p = rate_of_convergence(cg_best_x_seq_x1)
    print("Evaluation of rate of convergence for conjugate gradient method:")
    print(f"p mean = {np.abs(p).mean():.2f}")

Evaluation of rate of convergence for conjugate gradient method:

p mean = 2.18

Now let's work with the other functions and collect the data. 

1. Extended Powell

In [13]:
kmax = 5000

x0_array = np.random.randint(1,10,size = (10,1000))

In [14]:
kmean = 0 
grad_norm_mean = 0
grad_norm_min = 99999999999
grad_norm_max = -1
fx_mean = 0
fx_min = 99999999999
fx_max = -1

for point in x0_array: 
    sd_xk_ep, sd_fk_ep, sd_gradfk_norm_ep, sd_k_ep, sd_x_seq_ep, sd_bt_seq_ep = steepest_descent_bcktrck(point, 'Extended Powell', alpha0, kmax, tolgrad, c, rho, btmax, fin_diff, fd_type)
    print("Result of steepest descent method:")
    print("x0: ", point, " (length: ", len(point), ")")
    print("k: ", sd_k_ep)
    print("fk: ", sd_fk_ep[-1]) 
    print("gradfk: ", sd_gradfk_norm_ep[-1])
    print("\n")

    kmean += sd_k_ep
    grad_norm_mean += sd_gradfk_norm_ep[-1]
    if grad_norm_max < sd_gradfk_norm_ep[-1]: 
        grad_norm_max = sd_gradfk_norm_ep[-1]
    if grad_norm_min > sd_gradfk_norm_ep[-1]: 
        grad_norm_min = sd_gradfk_norm_ep[-1]
    fx_mean += sd_fk_ep[-1]
    if fx_max < sd_fk_ep[-1]: 
        fx_max = sd_fk_ep[-1]
    if fx_min > sd_fk_ep[-1]: 
        fx_min = sd_fk_ep[-1]
    

Result of steepest descent method:
x0:  [5 3 2 8 1 4 2 1 1 8 3 7 8 1 6 2 4 5 1 9 3 1 9 5 3 6 1 8 2 7 2 4 8 2 4 6 8
 6 1 7 5 1 7 3 4 6 6 8 1 3 5 9 5 4 7 2 1 2 2 5 8 6 9 6 1 5 8 4 7 7 8 3 5 4
 6 4 2 4 4 9 5 7 9 2 7 6 8 7 7 8 1 2 9 5 4 3 9 6 2 6 8 3 4 9 8 2 9 7 1 7 3
 2 2 6 3 7 7 8 9 1 5 8 1 5 2 6 2 9 5 3 9 9 9 5 4 9 9 2 1 6 7 8 1 8 7 6 4 7
 9 8 2 8 1 2 1 9 2 2 8 1 3 9 9 4 2 8 6 5 3 8 8 4 1 6 8 9 9 1 6 2 5 4 4 4 6
 5 5 2 5 3 8 9 3 5 1 6 1 8 2 3 4 6 5 2 3 1 4 7 7 7 8 8 2 5 5 1 7 1 3 9 4 4
 9 6 2 3 2 3 5 4 9 7 3 7 6 7 2 6 6 4 2 1 4 1 1 1 1 7 4 8 1 6 6 2 3 3 1 4 7
 3 1 3 2 2 1 8 3 4 6 7 7 8 8 2 7 1 2 3 8 3 7 4 9 6 9 9 4 9 6 7 3 8 9 1 2 6
 9 5 1 9 5 6 5 5 5 6 1 7 3 2 8 8 7 3 2 2 3 4 9 4 5 7 1 5 1 8 4 8 7 6 3 2 8
 4 1 3 9 3 1 4 4 4 5 2 2 1 4 7 3 1 3 5 7 6 3 9 7 1 1 3 3 2 9 2 1 5 6 2 9 1
 5 5 3 1 8 7 7 6 9 5 2 8 6 6 7 4 4 5 4 9 7 2 5 8 7 5 9 2 9 9 7 5 7 8 7 1 7
 3 2 6 1 8 2 1 8 7 7 4 7 9 4 8 2 6 3 6 3 1 3 9 9 1 8 4 5 8 3 3 4 1 2 4 1 9
 3 8 6 1 3 4 7 2 6 9 7 1 9 1 9 6 9 8 2 5 3 1 5 4 8 7 5 4 2 4

In [15]:
kmean = kmean / len(x0_array)
grad_norm_mean = grad_norm_mean / len(x0_array)
fx_mean = fx_mean / len(x0_array)

print("mean_of_k: ", kmean)

print("\n")
print("min_of_grad_norm: ", grad_norm_min)
print("mean_of_grad_norm: ", grad_norm_mean)
print("max_of_grad_norm: ", grad_norm_max)

print("\n")
print("min_of_fx: ", fx_min)
print("mean_of_fx: ", fx_mean)
print("max_of_fx: ", fx_max)

mean_of_k:  5000.0


min_of_grad_norm:  0.0037275764786322586
mean_of_grad_norm:  0.006661802243137288
max_of_grad_norm:  0.011756200323194063


min_of_fx:  0.000595576022060737
mean_of_fx:  0.0006006741091148918
max_of_fx:  0.0006052253469371171


In [16]:
kmean = 0 
grad_norm_mean = 0
grad_norm_min = 99999999999
grad_norm_max = -1
fx_mean = 0
fx_min = 99999999999
fx_max = -1

for point in x0_array: 
    sd_xk_ep, sd_fk_ep, sd_gradfk_norm_ep, sd_k_ep, sd_x_seq_ep, sd_bt_seq_ep = cgm_pol_rib(point, 'Extended Powell', alpha0, kmax, tolgrad, c, rho, btmax, fin_diff, fd_type)
    print("Result of conjugate gradient method:")
    print("x0: ", point, " (length: ", len(point), ")")
    print("k: ", sd_k_ep)
    print("fk: ", sd_fk_ep[-1]) 
    print("gradfk: ", sd_gradfk_norm_ep[-1])
    print("\n")
    
    kmean += sd_k_ep
    grad_norm_mean += sd_gradfk_norm_ep[-1]
    if grad_norm_max < sd_gradfk_norm_ep[-1]: 
        grad_norm_max = sd_gradfk_norm_ep[-1]
    if grad_norm_min > sd_gradfk_norm_ep[-1]: 
        grad_norm_min = sd_gradfk_norm_ep[-1]
    fx_mean += sd_fk_ep[-1]
    if fx_max < sd_fk_ep[-1]: 
        fx_max = sd_fk_ep[-1]
    if fx_min > sd_fk_ep[-1]: 
        fx_min = sd_fk_ep[-1]
    

Result of conjugate gradient method:
x0:  [5 3 2 8 1 4 2 1 1 8 3 7 8 1 6 2 4 5 1 9 3 1 9 5 3 6 1 8 2 7 2 4 8 2 4 6 8
 6 1 7 5 1 7 3 4 6 6 8 1 3 5 9 5 4 7 2 1 2 2 5 8 6 9 6 1 5 8 4 7 7 8 3 5 4
 6 4 2 4 4 9 5 7 9 2 7 6 8 7 7 8 1 2 9 5 4 3 9 6 2 6 8 3 4 9 8 2 9 7 1 7 3
 2 2 6 3 7 7 8 9 1 5 8 1 5 2 6 2 9 5 3 9 9 9 5 4 9 9 2 1 6 7 8 1 8 7 6 4 7
 9 8 2 8 1 2 1 9 2 2 8 1 3 9 9 4 2 8 6 5 3 8 8 4 1 6 8 9 9 1 6 2 5 4 4 4 6
 5 5 2 5 3 8 9 3 5 1 6 1 8 2 3 4 6 5 2 3 1 4 7 7 7 8 8 2 5 5 1 7 1 3 9 4 4
 9 6 2 3 2 3 5 4 9 7 3 7 6 7 2 6 6 4 2 1 4 1 1 1 1 7 4 8 1 6 6 2 3 3 1 4 7
 3 1 3 2 2 1 8 3 4 6 7 7 8 8 2 7 1 2 3 8 3 7 4 9 6 9 9 4 9 6 7 3 8 9 1 2 6
 9 5 1 9 5 6 5 5 5 6 1 7 3 2 8 8 7 3 2 2 3 4 9 4 5 7 1 5 1 8 4 8 7 6 3 2 8
 4 1 3 9 3 1 4 4 4 5 2 2 1 4 7 3 1 3 5 7 6 3 9 7 1 1 3 3 2 9 2 1 5 6 2 9 1
 5 5 3 1 8 7 7 6 9 5 2 8 6 6 7 4 4 5 4 9 7 2 5 8 7 5 9 2 9 9 7 5 7 8 7 1 7
 3 2 6 1 8 2 1 8 7 7 4 7 9 4 8 2 6 3 6 3 1 3 9 9 1 8 4 5 8 3 3 4 1 2 4 1 9
 3 8 6 1 3 4 7 2 6 9 7 1 9 1 9 6 9 8 2 5 3 1 5 4 8 7 5 4 2

In [17]:
kmean = kmean / len(x0_array)
grad_norm_mean = grad_norm_mean / len(x0_array)
fx_mean = fx_mean / len(x0_array)

print("mean_of_k: ", kmean)

print("\n")
print("min_of_grad_norm: ", grad_norm_min)
print("mean_of_grad_norm: ", grad_norm_mean)
print("max_of_grad_norm: ", grad_norm_max)

print("\n")
print("min_of_fx: ", fx_min)
print("mean_of_fx: ", fx_mean)
print("max_of_fx: ", fx_max)

mean_of_k:  4676.0


min_of_grad_norm:  5.919382004845522e-13
mean_of_grad_norm:  1.169201075604929e-08
max_of_grad_norm:  1.1625070468883891e-07


min_of_fx:  3.886238443961533e-18
mean_of_fx:  6.8301501364031145e-15
max_of_fx:  6.262611884658664e-14


2. Extended Rosenbrook

In [ ]:
kmean = 0 
grad_norm_mean = 0
grad_norm_min = 99999999999
grad_norm_max = -1
fx_mean = 0
fx_min = 99999999999
fx_max = -1

for point in x0_array: 
    sd_xk_er, sd_fk_er, sd_gradfk_norm_er, sd_k_er, sd_x_seq_er, sd_bt_seq_er = steepest_descent_bcktrck(point, 'Extended Rosenbrook', alpha0, kmax, tolgrad, c, rho, btmax, fin_diff, fd_type)
    print("Result of steepest descent method:")
    print("x0: ", point, " (length: ", len(point), ")")
    print("k: ", sd_k_er)
    print("fk: ", sd_fk_er[-1]) 
    print("gradfk: ", sd_gradfk_norm_er[-1])
    print("\n")
    
    kmean += sd_k_er
    grad_norm_mean += sd_gradfk_norm_er[-1]
    if grad_norm_max < sd_gradfk_norm_er[-1]: 
        grad_norm_max = sd_gradfk_norm_er[-1]
    if grad_norm_min > sd_gradfk_norm_er[-1]: 
        grad_norm_min = sd_gradfk_norm_er[-1]
    fx_mean += sd_fk_er[-1]
    if fx_max < sd_fk_er[-1]: 
        fx_max = sd_fk_er[-1]
    if fx_min > sd_fk_er[-1]: 
        fx_min = sd_fk_er[-1]
    

In [ ]:
kmean = kmean / len(x0_array)
grad_norm_mean = grad_norm_mean / len(x0_array)
fx_mean = fx_mean / len(x0_array)

print("mean_of_k: ", kmean)

print("\n")
print("min_of_grad_norm: ", grad_norm_min)
print("mean_of_grad_norm: ", grad_norm_mean)
print("max_of_grad_norm: ", grad_norm_max)

print("\n")
print("min_of_fx: ", fx_min)
print("mean_of_fx: ", fx_mean)
print("max_of_fx: ", fx_max)

In [ ]:
kmean = 0 
grad_norm_mean = 0
grad_norm_min = 99999999999
grad_norm_max = -1
fx_mean = 0
fx_min = 99999999999
fx_max = -1

for point in x0_array: 
    sd_xk_er, sd_fk_er, sd_gradfk_norm_er, sd_k_er, sd_x_seq_er, sd_bt_seq_er = cgm_pol_rib(point, 'Extended Rosenbrook', alpha0, kmax, tolgrad, c, rho, btmax, fin_diff, fd_type)
    print("Result of conjugate gradient method:")
    print("x0: ", point, " (length: ", len(point), ")")
    print("k: ", sd_k_er)
    print("fk: ", sd_fk_er[-1]) 
    print("gradfk: ", sd_gradfk_norm_er[-1])
    print("\n")
    
    kmean += sd_k_er
    grad_norm_mean += sd_gradfk_norm_er[-1]
    if grad_norm_max < sd_gradfk_norm_er[-1]: 
        grad_norm_max = sd_gradfk_norm_er[-1]
    if grad_norm_min > sd_gradfk_norm_er[-1]: 
        grad_norm_min = sd_gradfk_norm_er[-1]
    fx_mean += sd_fk_er[-1]
    if fx_max < sd_fk_er[-1]: 
        fx_max = sd_fk_er[-1]
    if fx_min > sd_fk_er[-1]: 
        fx_min = sd_fk_er[-1]
    

In [ ]:
kmean = kmean / len(x0_array)
grad_norm_mean = grad_norm_mean / len(x0_array)
fx_mean = fx_mean / len(x0_array)

print("mean_of_k: ", kmean)

print("\n")
print("min_of_grad_norm: ", grad_norm_min)
print("mean_of_grad_norm: ", grad_norm_mean)
print("max_of_grad_norm: ", grad_norm_max)

print("\n")
print("min_of_fx: ", fx_min)
print("mean_of_fx: ", fx_mean)
print("max_of_fx: ", fx_max)

3. Banded Trigonometric

In [ ]:
kmean = 0 
grad_norm_mean = 0
grad_norm_min = 99999999999
grad_norm_max = -1
fx_mean = 0
fx_min = 99999999999
fx_max = -1

for point in x0_array: 
    sd_xk_bt, sd_fk_bt, sd_gradfk_norm_bt, sd_k_bt, sd_x_seq_bt, sd_bt_seq_bt = steepest_descent_bcktrck(point, 'Banded Trigonometric', alpha0, kmax, tolgrad, c, rho, btmax, fin_diff, fd_type)
    print("Result of steepest descent method:")
    print("x0: ", point, " (length: ", len(point), ")")
    print("k: ", sd_k_bt)
    print("fk: ", sd_fk_bt[-1]) 
    print("gradfk: ", sd_gradfk_norm_bt[-1])
    print("\n")
    
    kmean += sd_k_bt
    grad_norm_mean += sd_gradfk_norm_bt[-1]
    if grad_norm_max < sd_gradfk_norm_bt[-1]: 
        grad_norm_max = sd_gradfk_norm_bt[-1]
    if grad_norm_min > sd_gradfk_norm_bt[-1]: 
        grad_norm_min = sd_gradfk_norm_bt[-1]
    fx_mean += sd_fk_bt[-1]
    if fx_max < sd_fk_bt[-1]: 
        fx_max = sd_fk_bt[-1]
    if fx_min > sd_fk_bt[-1]: 
        fx_min = sd_fk_bt[-1]

In [ ]:
kmean = kmean / len(x0_array)
grad_norm_mean = grad_norm_mean / len(x0_array)
fx_mean = fx_mean / len(x0_array)

print("mean_of_k: ", kmean)

print("\n")
print("min_of_grad_norm: ", grad_norm_min)
print("mean_of_grad_norm: ", grad_norm_mean)
print("max_of_grad_norm: ", grad_norm_max)

print("\n")
print("min_of_fx: ", fx_min)
print("mean_of_fx: ", fx_mean)
print("max_of_fx: ", fx_max)

In [ ]:
kmean = 0 
grad_norm_mean = 0
grad_norm_min = 99999999999
grad_norm_max = -1
fx_mean = 0
fx_min = 99999999999
fx_max = -1

for point in x0_array: 
    sd_xk_bt, sd_fk_bt, sd_gradfk_norm_bt, sd_k_bt, sd_x_seq_bt, sd_bt_seq_bt = cgm_pol_rib(point, 'Banded Trigonometric', alpha0, kmax, tolgrad, c, rho, btmax, fin_diff, fd_type)
    print("Result of steepest descent method:")
    print("x0: ", point, " (length: ", len(point), ")")
    print("k: ", sd_k_bt)
    print("fk: ", sd_fk_bt[-1]) 
    print("gradfk: ", sd_gradfk_norm_bt[-1])
    print("\n")
    
    kmean += sd_k_bt
    grad_norm_mean += sd_gradfk_norm_bt[-1]
    if grad_norm_max < sd_gradfk_norm_bt[-1]: 
        grad_norm_max = sd_gradfk_norm_bt[-1]
    if grad_norm_min > sd_gradfk_norm_bt[-1]: 
        grad_norm_min = sd_gradfk_norm_bt[-1]
    fx_mean += sd_fk_bt[-1]
    if fx_max < sd_fk_bt[-1]: 
        fx_max = sd_fk_bt[-1]
    if fx_min > sd_fk_bt[-1]: 
        fx_min = sd_fk_bt[-1]

In [ ]:
kmean = kmean / len(x0_array)
grad_norm_mean = grad_norm_mean / len(x0_array)
fx_mean = fx_mean / len(x0_array)

print("mean_of_k: ", kmean)

print("\n")
print("min_of_grad_norm: ", grad_norm_min)
print("mean_of_grad_norm: ", grad_norm_mean)
print("max_of_grad_norm: ", grad_norm_max)

print("\n")
print("min_of_fx: ", fx_min)
print("mean_of_fx: ", fx_mean)
print("max_of_fx: ", fx_max)